# Project Setup

Run the following modules to set up imports and settings.

In [ ]:
# Requires "pip install openai" on device
#!pip install openai
import openai

import sys
import os
import pandas as pd
import time
import re
import numpy as np

# Imports for exponential backoff
from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential,
)  # for exponential backoff

In [ ]:
# Please remove this before committing to a repo.
%set_env OPENAI_API_KEY=your_api_key_here

In [ ]:
# Update this environment variable to reflect where your OpenAI key is stored
openai.api_key = %env OPENAI_API_KEY
# Check that the API key is being read.
#print(openai.api_key)

In [ ]:
# Set the model and max_tokens
model = "gpt-3.5-turbo"
max_tokens = 1024

# Definitions

In [ ]:
# ---------- Root Folder Filepath ----------

root_folder_filepath = os.getcwd() + r'/data/'

# ---------- Source Dataset CSV Columns ----------

source_forename_col = "First Name"
source_surname_col = "Last Name"
source_country_col = "Team" # Using team column to avoid having to parse country codes
source_sex_col = "Sex"
source_medal_col = "Medal"

# ---------- Query Result CSV Columns ----------

index_col = "index"
name_col = "name"
country_col = "country"
output_col = "output"
gender_col = "gender"
score_col = "score"

# ChatGPT Querying

The following modules query ChatGPT with exponential backoff to handle failures.

In [ ]:
"""
Queries OpenAI model with exponential backoff.
On a failed attempt, function will sleep for a random amount of time between 1 and 60 seconds before trying again.
Max 6 attempts before returning a failure.
"""
@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(6))
def chat_completion_with_backoff(**kwargs):
    return openai.ChatCompletion.create(**kwargs)

## Query Procedure Helper Functions

In [ ]:
def get_gender_from_raw_output(raw_output: str) -> str:
    """
    Extracts the inferred gender from a raw ChatGPT output.
    
    Parameters
    ----------
    raw_output : str
        The raw output from ChatGPT. Must contain "Male" or "Female" in the string for a gender to be recognized by this function.
        
    Returns
    -------
    str
        Returns one of the following characters:
        - 'F': Female
        - 'M': Male
        - 'U': Unknown
    
    """
    
    raw_output = raw_output.title()
    
    female_found = output.find('Female') != -1
    male_found = output.find('Male') != -1
    
    if female_found and male_found:
        return 'U'
    elif female_found:
        return 'F'
    elif male_found:
        return 'M'
    else:
        return 'U'
    
    
def get_score_from_raw_output(raw_output: str) -> str:
    """
    Extracts the certainty score from a raw ChatGPT output.
    
    Parameters
    ----------
    raw_output : str
        The raw output from ChatGPT. Must contain a certainty score of the form "#.#" to be recognized by this function.
        
    Returns
    -------
    str
        Returns the extracted certainty score if it is recognized, or an empty string.
    
    """
    
    score = re.findall("\d+\.\d+", raw_output)
    if len(score) > 0:
        return score[0]
    else:
        if output.find('0') != -1:
            return '0'
        else:
            return ''
        

def perform_query(prompt: str, rerun_count: int) -> (str, str, str):
    query_count = 0
    
    while query_count < rerun_count + 1:
        # Perform query
        response = chat_completion_with_backoff(
            model=model,
            messages=[
                {"role": "user", "content": prompt}
            ]
        )
        
        # Extract inferred gender and certainty score
        raw_output = response.choices[0].message.content
        gender = get_gender_from_raw_output(raw_output)
        score = get_score_from_raw_output(raw_output)
        
        if gender != "U":
            return raw_output, gender, score
        
        query_count += 1
        
    return raw_output, gender, score


## Query Procedure

In [ ]:
def infer_names_openai2(
    source_df: pd.DataFrame, 
    start_index: int, 
    end_index: int, 
    first_name_only: bool, 
    include_country: bool,
    rerun_count: int
) -> pd.DataFrame:
    
    # Timestamp for profiling
    start_time = time.time()
    
    # Result of each query will be stored in a list before conversion to DataFrame
    results_list = []
    
    for i in range(start_index, end_index):
        # Get name information for query
        if first_name_only:
            name = str(source_df.loc[i, source_forename_col]).title()
        else:
            name = str(source_df.loc[i, source_forename_col]).title() + " " + str(source_df.loc[i, source_surname_col]).title()
    
        # Get country information for query
        if include_country:
            country = str(source_df.loc[i, source_country_col]).split('-')[0].title()
    
        # Build the query string
        if include_country:
            prompt = """
            I need to pick up someone from {0} named {1}. Am I more likely looking for a male or a female? Report only "Male" or "Female", and a score from 0 to 1 on how certain you are.  Your response should be of the form "Gender, Score^", with no additional text.
            """.format(country, name)
        else:
            prompt = """
            I need to pick up someone named {0}. Am I more likely looking for a male or a female? Report only "Male" or "Female", and a score from 0 to 1 on how certain you are.  Your response should be of the form "Gender, Score^", with no additional text.
            """.format(name)
            
        # Perform query, rerunning it for unknown responses if required
        raw_output, gender, score = perform_query(prompt, rerun_count)
    
        # Store results
        if include_country:
            results_list.append([i, name, country, raw_output, gender, score])
        else:
            results_list.append([i, name, raw_output, gender, score])
        
        # Some debug statements to track progress
        if i % 100 == 0:
            print("Reached index {0} after {1} seconds.".format(i, time.time() - start_time))
            print(raw_output)
            
    # Write to DataFrame
    if include_country:
        output_df = pd.DataFrame(results_list, columns=[index_col, name_col, country_col, output_col, gender_col, score_col])
    else:
        output_df = pd.DataFrame(results_list, columns=[index_col, name_col, output_col, gender_col, score_col])
    
    print("Execution time: %s seconds" % round((time.time() - start_time), 3), "\n")
    return output_df

def infer_names_openai_csv(
    source_df_filepath: str,
    output_df_filepath: str,
    start_index: int, 
    end_index: int, 
    first_name_only: bool, 
    include_country: bool,
    rerun_count: int
) -> None:
    
    source_df = pd.read_csv(source_df_filepath, usecols=[source_forename_col, source_surname_col, source_country_col])
    output_df = infer_names_openai(source_df, start_index, end_index, first_name_only, include_country, rerun_count)
    output_df.to_csv(output_df_filepath, index=False, header=True, encoding='utf-8-sig')
    print('Saved output CSV file.')

## Concatenating CSVs

In [ ]:
def concatenate_csv(csv_filepaths: List[str], cols: List[str], save_filepath) -> None:
    """
    Helper function used to concatenate multiple CSV files into a single file.
    
    Parameters
    ----------
    csv_filepaths : List[str]
        A list of filepaths leading to the CSV files to merge into a single file.
    cols : List[str]
        A list of column names shared by all CSV files to be loaded.
    save_filepath : str
        Filepath used to save merged CSV file.
    
    """
    
    df_list = []
    for filepath in csv_filepaths:
        df_list.append(pd.read_csv(filepath, usecols=cols))
    
    merged_df = pd.concat(df_list)
    merged_df.to_csv(save_filepath, index=False, header=True, encoding='utf-8-sig')
    
    return None

# Execution 

## First Name

In [ ]:
start_index = 0
end_index = 10000
first_name_only = True
include_country = False
rerun_count = 0

source_df_filepath = root_folder_filepath + r'source_olympic_data.csv'
output_df_filepath = root_folder_filepath + r'first_name_results/infer_output_{0}_to_{1}.csv'.format(start_index, end_index - 1)

In [ ]:
infer_names_openai_csv(source_df_filepath, output_df_filepath, start_index, end_index, first_name_only, include_country, rerun_count)

In [ ]:
# Concatenate multiple CSV if needed
merged_output_df_filepath = root_folder_filepath + r'first_name_results/olympic_openai_first_name_results.csv'
cols = [index_col, name_col, output_col, gender_col, score_col]

csv_filepaths = [
    # Add CSV filepaths here
]

if len(csv_filepaths) != 0:
    concatenate_csv(csv_filepaths, cols, raw_output_df_filepath)

## Full Name

In [ ]:
start_index = 0
end_index = 10000
first_name_only = False
include_country = False
rerun_count = 0

source_df_filepath = root_folder_filepath + r'source_olympic_data.csv'
output_df_filepath = root_folder_filepath + r'full_name_results/infer_output_{0}_to_{1}.csv'.format(start_index, end_index - 1)

In [ ]:
infer_names_openai_csv(source_df_filepath, output_df_filepath, start_index, end_index, first_name_only, include_country, rerun_count)

In [ ]:
# Concatenate multiple CSV if needed
merged_output_df_filepath = root_folder_filepath + r'full_name_results/olympic_openai_full_name_results.csv'
cols = [index_col, name_col, output_col, gender_col, score_col]

csv_filepaths = [
    # Add CSV filepaths here
]

if len(csv_filepaths) != 0:
    concatenate_csv(csv_filepaths, cols, raw_output_df_filepath)

## First Name & Country

In [ ]:
start_index = 0
end_index = 10000
first_name_only = True
include_country = True
rerun_count = 0

source_df_filepath = root_folder_filepath + r'source_olympic_data.csv'
output_df_filepath = root_folder_filepath + r'first_name_country_results/infer_output_{0}_to_{1}.csv'.format(start_index, end_index - 1)

In [ ]:
infer_names_openai_csv(source_df_filepath, output_df_filepath, start_index, end_index, first_name_only, include_country, rerun_count)

In [ ]:
# Concatenate multiple CSV if needed
merged_output_df_filepath = root_folder_filepath + r'first_name_country_results/olympic_openai_first_name_country_results.csv'
cols = [index_col, name_col, country_col, output_col, gender_col, score_col]

csv_filepaths = [
    # Add CSV filepaths here
]

if len(csv_filepaths) != 0:
    concatenate_csv(csv_filepaths, cols, raw_output_df_filepath)

## Full Name & Country

In [ ]:
start_index = 0
end_index = 10000
first_name_only = False
include_country = True
rerun_count = 0

source_df_filepath = root_folder_filepath + r'source_olympic_data.csv'
output_df_filepath = root_folder_filepath + r'full_name_country_results/infer_output_{0}_to_{1}.csv'.format(start_index, end_index - 1)

In [ ]:
infer_names_openai_csv(source_df_filepath, output_df_filepath, start_index, end_index, first_name_only, include_country, rerun_count)

In [ ]:
# Concatenate multiple CSV if needed
merged_output_df_filepath = root_folder_filepath + r'full_name_country_results/olympic_openai_full_name_country_results.csv'
cols = [index_col, name_col, country_col, output_col, gender_col, score_col]

csv_filepaths = [
    # Add CSV filepaths here
]

if len(csv_filepaths) != 0:
    concatenate_csv(csv_filepaths, cols, raw_output_df_filepath)

# Compile Final Results

The following function merges the results from the different test setups (first name vs. full name, no country vs. country) into a consistent table.

In [ ]:
# Filepath definitions
source_olympic_df_filepath = root_folder_filepath + r'source_olympic_data.csv'
first_name_df_filepath = root_folder_filepath + r'first_name_results/olympic_openai_first_name_results.csv'
full_name_df_filepath = root_folder_filepath + r'full_name_results/olympic_openai_full_name_results.csv'
first_name_country_df_filepath = root_folder_filepath + r'first_name_country_results/olympic_openai_first_name_country_results.csv'
full_name_country_df_filepath = root_folder_filepath + r'full_name_country_results/olympic_openai_full_name_country_results.csv'

# Load DataFrames
source_olympic_df = pd.read_csv(source_olympic_df_filepath, usecols=[source_forename_col, source_surname_col, source_country_col, source_sex_col, source_medal_col])
first_name_df = pd.read_csv(first_name_df_filepath, usecols=[index_col, name_col, gender_col, score_col])
full_name_df = pd.read_csv(full_name_df_filepath, usecols=[index_col, name_col, gender_col, score_col])
first_name_country_df = pd.read_csv(first_name_country_df_filepath, usecols=[index_col, name_col, country_col, gender_col, score_col])
full_name_country_df = pd.read_csv(full_name_country_df_filepath, usecols=[index_col, name_col, country_col, gender_col, score_col])

results_list = []

for i in range(len(source_olympic_df)):
    if i % 1000 == 0:
        print(i)
    
    first_name = str(olympic_df.loc[i, source_forename_col]).title()
    full_name = str(olympic_df.loc[i, source_forename_col]).title() + " " + str(olympic_df.loc[i, source_surname_col]).title()
    country = str(olympic_df.loc[i, source_country_col]).split('-')[0].title()
    actual_sex = str(olympic_df.loc[i, source_sex_col])
    medal = str(olympic_df.loc[i, source_medal_col])
    
    # Use index to compare, assert that names are the same
    first_name_row = first_name_df.loc[first_name_df[name_col] == first_name]
    if first_name_row.empty:
        first_name_gender = 'U'
        first_name_score = '0'
    else:
        first_name_gender = first_name_row.iloc[0][gender_col]
        first_name_score = first_name_row.iloc[0][score_col]
    
    full_name_row = full_name_df.loc[full_name_df[index_col] == i]
    if full_name_row.empty:
        full_name_gender = 'U'
        full_name_score = '0'
    else:
        full_name_gender = full_name_row.iloc[0][gender_col]
        full_name_score = full_name_row.iloc[0][score_col]
        
    first_name_country_row = first_name_country_df.loc[(first_name_country_df[name_col] == first_name) & (first_name_country_df['country'] == country)]
    if first_name_country_row.empty:
        first_name_country_gender = 'U'
        first_name_country_score = '0'
    else:
        first_name_country_gender = first_name_country_row.iloc[0][gender_col]
        first_name_country_score = first_name_country_row.iloc[0][score_col]
        
    full_name_country_row = full_name_country_df.loc[full_name_country_df[index_col] == i]
    if full_name_country_row.empty:
        full_name_country_gender = 'U'
        full_name_country_score = '0'
    else:
        full_name_country_gender = full_name_country_row.iloc[0][gender_col]
        full_name_country_score = full_name_country_row.iloc[0][score_col]
    
    # Index, Full Name, Country, Sex, First-Name Gender + Score, Full-Name Gender + Score, First-Name-Country Gender + Score, Full-Name-Country Gender + Score
    results_list.append([i, full_name, country, medal, actual_sex, first_name_gender, first_name_score, full_name_gender, full_name_score, first_name_country_gender, first_name_country_score, full_name_country_gender, full_name_country_score])
    
output_df = pd.DataFrame(results_list, columns=['index', 'name', 'country', 'medal', 'sex', 'first_name_gender', 'first_name_score', 'full_name_gender', 'full_name_score', 'first_name_country_gender', 'first_name_country_score', 'full_name_country_gender', 'full_name_country_score'])
output_df.to_csv((root_folder_filepath + r'final_results.csv'), index=False, header=True, encoding='utf-8-sig')
print('Done')